# TP N°1 : Algèbre - Remise à niveau - Question 4
## Romain DOMART
### 09/10/2018

L'objectif de cette question est de reproduire les valeurs trouvées dans les tableaux 3.2 et 3.3 disponibles dans *Hastie, Trevor, Tibshirani, Robert and Friedman, Jerome. The Elements of Statistical Learning. New York, NY, USA: Springer New York Inc., 2001.*

# Initialisation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
from sklearn.preprocessing import StandardScaler, scale
from sklearn.linear_model import LinearRegression
from sklearn import metrics 
from sklearn.model_selection import train_test_split
np.set_printoptions(precision=3)
pd.options.display.float_format = '{:,.3f}'.format
#import warnings
#warnings.filterwarnings('ignore')

In [2]:
def append_nones(length, list_):
    """
    Appends Nones to list to get length of list equal to `length`.
    If list is too long raise AttributeError
    """
    diff_len = length - len(list_)
    if diff_len < 0:
        raise AttributeError('Length error list is too long.')
    return list_ + [None] * diff_len



In [4]:
data = pd.read_csv("https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data",delimiter = '\t+',index_col=0)
resultats = pd.DataFrame(index=["Intercept","lcavol","lweight","age","lbph","svi","lcp","gleason","pgg45"])
zScore = pd.DataFrame(index=["Intercept","lcavol","lweight","age","lbph","svi","lcp","gleason","pgg45"])
error = pd.DataFrame(index = ["MSE","Std Error"])

/Users/romaindomart/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


## Analyse exploratoire des données

In [ ]:
data.head() #Permet d'afficher si les données ont été correctement importées

In [ ]:
data.describe() # PPermet d'avoir des premieres informations sur le jeu de données.

### Étude des corrélations entre variables

In [ ]:
f = plt.figure(figsize=(10,5))
sns.heatmap(data.corr(),cmap="viridis_r",annot=True)
plt.title("Heatmap of correlation between variables")
f.tight_layout()



In [ ]:
sns.pairplot(data)

### Conclusion de l'analyse exploratoire

On peut remarquer plusieurs choses : 
* la variables svi est un variable qualitative prenant deux valeurs $(0,1)$
* la variable gleason ne prend que 4 valeurs $(6,7,8,9)$. 
* il y a une correlation entre les variables **lcavol** et **lpsa** ainsi qu'entre les variables **lcp** et **lpsa**. Or les variables **lcavol** et **lcp** sont elles aussi corrélées entre elles. Ainsi on peut tout de suite savoir que pour un modèle de regression prenant la variable **lcavol** en paramètre, l'ajout supplémentaire de **lcp** n'apportera que peu de nouvelles informations. 

# Formulation du problème de regression

Soit $n=93$, le nombre d'observations et $p=8$ le nombre de variables : 

On définit 
$$
\begin{array}{cc}
\mathbf{X} \in \mathcal{M}_{n,p} & \mathbf{y} \in \mathcal{M}_{n,1}
\end{array}
$$






In [7]:
X = data.drop(["lpsa","train"],axis=1).values
y = data["lpsa"].values

Centrage - Réduction de l'ensemble des valeurs Cette étape doit être omise dans une démonstration rigoureuse. On y a recours pour retrouver les résultats du livre

In [8]:
scaler = StandardScaler()
X=scaler.fit(X).transform(X)


### Création des sets d'apprentissage et de test

In [9]:
X_train = X[data["train"]=="T"]
X_test= X[data["train"]=="F"]
y_train= y[data["train"]=="T"]
y_test = y[data["train"]=="F"]

In [10]:
y_train_mean = y_train.mean()
y_test_mean = y_test.mean()

# Least Squares
On définit la fonction $f$ telle que 


$$f:
\left|
  \begin{array}{rcl}
    {M}_{n,p} & \longrightarrow & {M}_{n,1} \\
    \mathbf{X} & \longmapsto &   \mathbf{X}\beta + \beta_{0} \\
  \end{array}
\right.
$$

avec $\beta_{0}^{\ast}$ et $\beta^{\ast}$ les paramètres *réels* du modèle tel que $\mathbf{y}=f\left(\mathbf{X},\beta_{0}^{\ast},\beta^{\ast}\right)$.

On définit ensuite $\hat{\mathbf{y}}$, $\hat{\beta}_{0}$ et $\hat{\beta}$ les prédictions et paramètres *estimés* par le modèle de régression tels que $\hat{\mathbf{y}} = f\left(\mathbf{X},\hat{\beta}_{0}, \hat{\beta}\right)$.

Enfin on définit $\beta_{0}$ et $\beta$ des paramètres *libres*.

\begin{align}
\hat{\beta}
&=\text{arg}\min_{\beta} \left( \sum_{i=1}^{n}\left(y_{i}-f\left(x_{i}\right)\right)^{2}\right) \\
&=\text{arg}\min_{\beta} \left( \sum_{i=1}^{n}\left(y_{i}-\beta_{0}-\sum_{j=1}^{p}\left(x_{i,j}\beta_{j}\right)\right)^{2} \right) \\
\end{align}


On définit :
\begin{align}
\mathbf{X}_{a} =
\begin{bmatrix} 
1& x_{1,1} &  \dots & x_{1,p} \\
\vdots &\vdots & \ddots & \vdots \\
1&x_{1,p} &    \dots    & x_{n,p} 
    \end{bmatrix} & 
    & 
    \mathbf{\beta}_{a} = \begin{bmatrix} 
\beta_{0}\\
\beta_{1}\\
\vdots \\
 \beta_{p} 
    \end{bmatrix}
\end{align}


On obtient $\beta$ de la forme : 

\begin{align}
\hat{\beta}&=\text{arg}\min_{\beta} \left( \|\mathbf{y}-\mathbf{X_{a}\beta_{a}\|}^{2} \right) 
\end{align}


### Calcul de la regression des moindres carrés - Solution analytique

In [ ]:
n,p = X_train.shape
d = np.ones((n,1))
#Création de Xa
Xa = np.concatenate((d,X_train), axis = 1) 
#Résolution du problème des moindres carrés
beta_ls = np.linalg.solve(np.matmul(Xa.T,Xa),np.matmul(Xa.T,y_train))
print('Beta Least Square from analytics: \n', beta_ls)

#Prédictions de y
n,p = X_test.shape
d = np.ones((n,1))
Xa_test = np.concatenate((d,X_test), axis = 1)
predictions_analytics_ls = np.matmul(Xa_test,beta_ls)
print('\n\nPredictions from analytics: \n', predictions_analytics_ls)

###  *Optionnel - Calcul de la regression des moindres carrés - Solution du package sklearn*

In [ ]:
from sklearn.linear_model import LinearRegression
ls = LinearRegression()
ls.fit(X_train,y_train)
print('Beta Least Square from package () : \n',ls.intercept_, ls.coef_)
predictions_package_ls = ls.predict(X_test)
print('\n\nPredictions from package: \n', predictions_package_ls)

### Calcul de la regression des moindres carrés - Calcul des erreurs

In [ ]:
print('MSE analytics:' , metrics.mean_squared_error(y_test,predictions_analytics_ls))
print('MSE package:' , metrics.mean_squared_error(y_test,predictions_package_ls))

In [ ]:
print ('Standard error analytics: ', np.sqrt(sum((y_test-predictions_analytics_ls)**2)/y_test.shape[0]))
print ('Standard error package: ', np.sqrt(sum((y_test-predictions_package_ls)**2)/y_test.shape[0]))

In [ ]:
resultats["Least Square"] = beta_ls
error["Least Square"] = [metrics.mean_squared_error(y_test,predictions_analytics_ls),np.sqrt(sum((y_test-predictions_analytics_ls)**2)/y_test.shape[0])]

# Calcul du meilleur subset - Z score

Rappelons tout d'abord l'expression générale du z-score
\begin{align}
    z &= \frac{x-\bar{x}}{\sqrt{\text{Var}(x)}}
\end{align}
Avec $\bar{x}$ la moyenne de $x$ et $\sqrt{\text{Var}(x)}$ l'écart-type de x.

Notons l'expression du z-score dans le cas de $\hat{\beta_{a}}$
\begin{align}
    z &= \frac{\hat{\beta_{a}}-\bar{\hat{\beta_{a}}}}{\sqrt{\text{Var}(\hat\beta_{a})}}
\end{align}

\begin{align}
\mathbf{E}\left(\hat{\beta_{a}}\right) &= \mathbf{E}\left(\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X}^{T}\mathbf{y}\right) \\
&= \left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\mathbf{E}\left(\mathbf{y}\right) \\
&= \left(\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)\beta_{a}^{\ast}+\mathbf{E}\left(\mathbf{\epsilon}\right)\\
&= \mathbf{I}\beta_{a}^{\ast}\\
&= \beta_{a}^{\ast}
\end{align}




Et 
\begin{align}
    \text{Var}\left(\hat{\beta_{a}}\right) &= \text{Var}\left(\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\mathbf{y}\right) \\
    &= \left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\text{Var}\left(\mathbf{y}\right)\mathbf{X_{a}}\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\\
    &= \left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X}^{T}\text{Var}\left(\mathbf{X_{a}}\beta_{a}^{\ast}+\mathbf{\epsilon}\right)\mathbf{X_{a}}\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\\
    &= \left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\text{Var}\left(\mathbf{\epsilon}\right)\mathbf{X_{a}}\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1} \\
    &= \sigma^2\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}\mathbf{X_{a}}^{T}\mathbf{X_{a}}\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1} \\
    &= \sigma^2\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}
\end{align}


Pour tester l'hypothèse qu'un certain coefficient ${\beta_{a_j}^{\ast}} = 0$ pour $j=1 \cdot p$, on introduit donc le $Z-score$ tel que:
\begin{align}
    z_j &= \frac{\hat{\beta_{a_{j}}}}{\hat{\sigma}\sqrt{v_j}} 
\end{align}

avec $v_j$ le $j^e$ élément de la diagonale de $\left(\mathbf{X_{a}}^{T}\mathbf{X_{a}}\right)^{-1}$

In [ ]:
n,p = X_train.shape
V_ls = np.matmul(Xa.T,Xa) # Calcul de V
V_ls_inv = np.linalg.inv(V_ls) # Inverse de V
v = np.diagonal(V_ls_inv) # Diagnonale de V

In [ ]:
#Calcul de sigma
sigma = 0
for i in range (n):
    sigma = sigma+(y_train[i]-np.matmul(Xa,beta_ls)[i])**2
sigma = (1/(n-p-1))*sigma

In [ ]:
#Calcul des z
z=[]
for i in range(beta_ls.shape[0]):
    zi = beta_ls[i]/np.sqrt(sigma*v[i])
    z.append(zi)
zScore["Coefficient"]=beta_ls
zScore["Std Error"] = sigma*v
zScore["Z-value"]=z

In [ ]:
print('Tableau 3.2\n',error)

Les Z-values pour Intercept, lcavol et lweight sont les plus élevés. Ce sont les variables pour lesquels l'hypothèse ${\beta_{a_j}^{\ast}} = 0$ peut être rejetée avec le plus de certitude. 

# *Optionnel - Calcul du meilleur subset - Suite*

Le calcul d'une regression sur l'ensemble des variables n'est pas avantageux dans le sens où l'addition de variables supplémentaires au modèle ne peut apporter que très peu d'informations supplémentaires. Il nous faut ainsi sélectionner le meilleur compromis entre le nombre de variables et le taux d'erreur de la regression. 


### *Recherche exhaustive des subsets*

Ici nous allons effectuer une recherche exhaustive sur l'ensemble des combinaisons de variables possibles pour le calcul de la regression puis calculer deux indicateurs : l'AIC et le BIC afin de déterminer le tuple de variables à considérer dans notre regresssion.

Pour cela, et comme ce n'est pas le sujet du TP, nous allons utiliser le package statsmodel pour effectuer la régression des moindres carrés et determiner directement ces deux indicateurs avant de réefectuer le calcul de la regression avec le subset choisi par l'énoncé de manière analytique

Note : Le code python utilisé ci-dessous n'est pas optimisé.


In [ ]:
#Calcul exhaustif des combinaisons de variables

from itertools import chain, combinations
l=list(chain(*map(lambda x: combinations(range(8),x),range(0,len(range(8))+1))))
l = l[1:]


In [ ]:
#Calcul des BIC / AIC avec le package statsmodel
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
listofaic = []
listofbic=[]
for x in l : 
    regr = OLS(y_train,add_constant(X_train[:,x])).fit()
    listofaic.append(regr.aic)
    listofbic.append(regr.bic)


daic = pd.DataFrame([l,listofaic]).T.sort_values(by=[1]).reset_index().drop("index",axis=1)
dbic =pd.DataFrame([l,listofbic]).T.sort_values(by=[1]).reset_index().drop("index",axis=1)




### *Calculs des meilleurs subsets*
On peut ainsi déterminer les meilleurs subsets sur lesquels effectuer la regression des moindres carrés. 

*0 = lcavol \[...\] 7 = pgg45*

In [ ]:
print ("Critère BIC \n", dbic.head(5))
print ("Critère AIC \n",daic.head(5))

In [ ]:
print("Critère BIC pour les tuples de taille 2\n",dbic[dbic[0].str.len()==3].head(5))
print("Critère AIC pour les tuples de taille 2\n",daic[daic[0].str.len()==3].head(5))



On retrouve les résultats obtenus avec le calcul des $Z-scores$

# Best Subset
### Calcul de la regression des moindres carrés pour le meilleur subset $(0,1)$ - Solution analytique




In [ ]:
X_train = X[data["train"]=="T"][:,[0,1]]
X_test= X[data["train"]=="F"][:,[0,1]]
y_train= y[data["train"]=="T"]


In [ ]:
n,p = X_train.shape
d = np.ones((n,1))
Xa = np.concatenate((d,X_train), axis = 1)
beta_ls = np.linalg.solve(np.matmul(Xa.T,Xa),np.matmul(Xa.T,y_train)) #Résolution du problème
print('Beta Least Square from analytics: \n', beta_ls)
n,p = X_test.shape
d = np.ones((n,1))
Xa_test = np.concatenate((d,X_test), axis = 1)
predictions_analytics_ls = np.matmul(Xa_test,beta_ls)
print('\n\nPredictions from analytics: \n', predictions_analytics_ls)

In [ ]:
print('MSE analytics:' , metrics.mean_squared_error(y_test,predictions_analytics_ls))
print ('Standard error analytics: ', np.sqrt(sum((y_test-predictions_analytics_ls)**2)/y_test.shape[0]))

In [ ]:
resultats["Best Subset"] = append_nones(resultats.shape[0],list(beta_ls))
error["Best Subset"] = [metrics.mean_squared_error(y_test,predictions_analytics_ls),np.sqrt(sum((y_test-predictions_analytics_ls)**2)/y_test.shape[0])]



# Ridge

$$f:
\left|
  \begin{array}{rcl}
    {M}_{n,p} & \longrightarrow & {M}_{n,1} \\
    X & \longmapsto &  \mathbf{X}\beta + \beta_{0} +\lambda\beta \\
  \end{array}
\right.
$$

On définit $\beta_{0}^{\ast}$ et $\beta^{\ast}$ les paramètres *réels* du modèle tel que $\mathbf{y}=f\left(\mathbf{X},\beta_{0}^{\ast},\beta^{\ast}\right)$.

On définit ensuite $\hat{\mathbf{y}}$, $\hat{\beta}_{0}$ et $\hat{\beta}$ les prédictions et paramètres *estimés* par le modèle de régression tels que $\hat{\mathbf{y}} = f\left(\mathbf{X},\hat{\beta}_{0}, \hat{\beta}\right)$.

Enfin on définit $\beta_{0}$ et $\beta$ des paramètres *libres*.


\begin{align}
\hat{\beta}&=\text{arg}\min_{\beta} \left( \sum_{i=1}^{n}\left(y_{i}-\beta_{0}-\sum_{j=1}^{p}\left(x_{i,j}\beta_{j}\right)\right)^{2} + \lambda\sum_{j=1}^{p}\beta_{j}^{2}\right) 
\end{align}
Avec $\lambda\geq 0$ fixé
Centrons maintenant la matrice $\mathbf{X}$ tel que $\mathbf{X_c} = \mathbf{X}-\mathbf{\bar{X}}$. Ainsi on a :

\begin{align}
\hat{\beta}&=\text{arg}\min_{\beta} \left( \sum_{i=1}^{n}\left(y_{i}-\sum_{j=1}^{p}\left(x_{c_{i,j}}\beta_{j}\right)\right)^{2} + \lambda\sum_{j=1}^{p}\beta_{j}^{2}\right) \\
&=\text{arg}\min_{\beta} \left( \|{\mathbf{y}-\mathbf{X_c\beta}\|}^{2} + \|{\lambda\beta}\|\right) 
\end{align}

On trouve $\beta$ de la forme : 

\begin{align}
    \beta &= \left(\mathbf{X_{c}}^{T}\mathbf{X_{c}} - \lambda\mathbf{I}\right)^{-1}\mathbf{X_{c}}^{T}\mathbf{y}
\end{align}

### Il faut tout d'abord centrer nos données d'apprentissage et de test. 

Ici, afin de retrouver les valeurs du tableau à reproduire, on centre aussi les valeurs de $\mathbf{y}$.

In [11]:
X_train = X[data["train"]=="T"]
X_test= X[data["train"]=="F"]
y_train= y[data["train"]=="T"]
y_test = y[data["train"]=="F"]

In [12]:
scaler = StandardScaler(with_std=False)
X_train=scaler.fit(X_train).transform(X_train)
X_test=scaler.fit(X_test).transform(X_test)
y_train = y_train-y_train_mean
y_test = y_test - y_test_mean

### Calcul du $\lambda$ 

On définit tout d'abord la décomposition singulière de $\mathbf{X}$ telle que : 
\begin{align}
\mathbf{X}_{c} 
&= \mathbf{U}\mathbf{D}\mathbf{V}^{T}
\end{align}

Avec : 
$$
\begin{array}{cc}
\mathbf{D} \in \mathcal{M}_{n,p} & \mathbf{U} \in \mathcal{M}_{n,n} & \mathbf{V} \in \mathcal{M}_{p,p}
\end{array}
$$
L'énoncé nous indique : 

\begin{align}
df\left(\lambda\right) &= \text{tr}\left(\mathbf{X_{c}}^{T}\mathbf{X_{c}} - \lambda\mathbf{I}\right)^{-1}\mathbf{X_{c}}^{T} \\
&= \sum_{j=1}^{p}\left(\frac{d_{j}^{2}}{d_{j}^{2}+\lambda}\right)\\
&= 5  
\end{align}

Avec $d_{1\dots p}$ les valeurs portées par la diagonale de $\mathbf{D}$



In [26]:
#Calcul de lambda avec le module scipy. On determine le minimum de la fonction df.

import scipy

U,d,V = scipy.linalg.svd(X_train)
import numpy as np
from scipy.optimize import fmin
import math

def df(x):
    df=0
    for i in range(d.shape[0]):
        df=df+(d[i]*d[i])/(d[i]*d[i]+x)
    return abs(df-5)   
array = np.array([0,df(0)])
lamb = fmin(df,0)

print('Valeur de lambda telle que df(lambda)=5 :', lamb)


Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 34
         Function evaluations: 68
Valeur de lambda telle que df(lambda)=5 : [24.249]


### Calcul de la regression Ridge - Solution analytique

In [ ]:
n,p = X_test.shape
I = np.identity(p)
Xtemp = np.matmul(X_train.T,X_train)
Xtemp = np.add(Xtemp,lamb*I)
beta_r = np.linalg.solve(Xtemp,np.matmul(X_train.T,y_train))


print('Beta Ridge from analytics : \n',y_train_mean, beta_r)
predictions_analytics_r = y_test.mean() + np.matmul(X_test,beta_r)
print('\n\nPredictions from analytics : \n', predictions_analytics_r)

###  *Optionnel - Calcul de la regression des moindres carrés - Solution du package sklearn*

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=lamb)
ridge.fit(X_train,y_train)
print('Beta Ridge from package () : \n',ridge.intercept_, ridge.coef_)
predictions_package_r = ridge.predict(X_test)
print('\n\nPredictions from package : \n', predictions_package_r)

In [ ]:
a = np.arange(0.1,100,0.01)

from sklearn.linear_model import RidgeCV
ridgecv = RidgeCV(alphas = tuple(a))
ridgecv.fit(X_train,y_train)
print('\n\nBeta ridge from CV : ', ridgecv.intercept_, ridgecv.coef_)
print('\n\nAlpha from CV : ', ridgecv.alpha_)
predictions_crossval_r = ridgecv.predict(X_test)
print('\n\nPredictions from CV : \n', predictions_crossval_r)

In [ ]:
print('MSE analytics:' , metrics.mean_squared_error(y_test,predictions_analytics_r))
print('MSE package:' , metrics.mean_squared_error(y_test,predictions_package_r))
print('MSE crossval:' , metrics.mean_squared_error(y_test,predictions_crossval_r))

In [ ]:
print ('Standard error analytics: ', np.sqrt(sum((y_test-predictions_analytics_r)**2)/y_test.shape[0]))
print ('Standard error package: ', np.sqrt(sum((y_test-predictions_package_r)**2)/y_test.shape[0]))
print ('Standard error crossval: ', np.sqrt(sum((y_test-predictions_crossval_r)**2)/y_test.shape[0]))

In [ ]:
b=beta_r.tolist()
b.insert(0,y_train_mean)

In [ ]:
resultats["Ridge"]=b
error["Ridge"] = [metrics.mean_squared_error(y_test,predictions_analytics_r),np.sqrt(sum((y_test-predictions_analytics_r)**2)/y_test.shape[0])]

# Conclusion

In [27]:
print('Tableau 3.2')
display(zScore)
print('\n\nTableau 3.3')
display(resultats)
display(error)

Tableau 3.2


""
Intercept
lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45




Tableau 3.3


""
Intercept
lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45


""
MSE
Std Error
